# 네이버 지식인 크롤링
- keyword 임의설정
- 당일 데이터만 수집

In [16]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from urllib.parse import quote
from tqdm import tqdm
from datetime import datetime, timedelta

## 오늘 날짜만 검색

In [17]:
KEYWORD = '대선'
TODAY = datetime.today().strftime("%Y.%m.%d.")
print(TODAY)

2021.04.20.


In [19]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
base_url = "https://www.naver.com/"
driver.get(base_url)
time.sleep(1)

# 지식iN 으로 이동
nav = driver.find_element_by_id('NM_FAVORITE')
lis = nav.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
lis[3].click()

# 키워드 검색 후 최신순 정렬
search_area = driver.find_element_by_class_name('search_area')
search_area.find_element_by_tag_name('input').send_keys(KEYWORD)    # 키워드 입력
driver.find_element_by_class_name('search_btn').click()     # 검색
driver.find_element_by_class_name('option_sort').find_elements_by_tag_name('li')[1].click()     # '최신순' 클릭
time.sleep(1)

# 기간 검색: TODAY
driver.find_element_by_id('au_date_select').click()
time.sleep(1)
from_date = driver.find_element_by_id('sel_from_date')
to_date = driver.find_element_by_id('sel_to_date')
# 시작날짜, 완료날짜 입력
actions = ActionChains(driver).double_click(from_date).send_keys(TODAY).double_click(to_date).send_keys(TODAY)
actions.perform()
driver.find_element_by_xpath('/html/body/div[3]/div/ul/li[4]/input[3]').click()

# 전체 페이지수 수집
while True:
    pagination = driver.find_element_by_class_name('s_paging')
    pages = pagination.find_elements_by_tag_name('a')
    tmp = []
    for page in pages:
        tmp.append(page.text)
    if '다음페이지' in tmp:
        pages[-1].click()
        time.sleep(2)
    else:
        NUM_OF_PAGE = int(pagination.text.split()[-1])
        break
print("전체 페이지 수: ", NUM_OF_PAGE)
driver.close()

전체 페이지 수:  1


In [20]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
ans_url_list = []; total_ans = 0

# 질문들의 url 수집
for page in range(NUM_OF_PAGE):
    kin_url = f'https://kin.naver.com/search/list.nhn?sort=date&query={quote(KEYWORD)}'
    period_url = f'&period={TODAY}%7C{TODAY}&section=kin'
    page_url = f'&page={page+1}'
    url = kin_url + period_url + page_url
    driver.get(url)
    time.sleep(1)
    #print(f"{page+1} 페이지")

    ul = driver.find_element_by_xpath('//*[@id="s_content"]/div[3]/ul')
    lis = ul.find_elements_by_tag_name('li')
    for li in lis:
        a_tag = li.find_element_by_tag_name('a')
        ans_url_list.append(a_tag.get_attribute('href'))

print(f'총 {len(ans_url_list)}개의 질문')

# 각 질문의 답변 개수
for i, ans_url in enumerate(ans_url_list):
    driver.get(ans_url)
    time.sleep(0.5)
    ans_cnt = int(driver.find_element_by_class_name('_answerCount.num').text)
    total_ans += ans_cnt
    print(f'{i+1}번째 : {ans_cnt}')
print("----------------------------------")
print(f'Total 답변 개수: {total_ans}')

driver.close()

총 4개의 질문
1번째 : 3
2번째 : 1
3번째 : 1
4번째 : 1
----------------------------------
Total 답변 개수: 6


In [21]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
answer_text_list = []

for ans_url in tqdm(ans_url_list):
    driver.get(ans_url)
    time.sleep(1)
    more = driver.find_element_by_xpath('//*[@id="nextPageButton"]')
    disp = more.get_attribute('style')      # 더보기 할 답변이 남아있지 않은 경우 "style:None" 이 존재한다
    try:
        # 더보기 할 답변이 남지 않을 때까지 while문을 돌린다
        while disp:     
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(1)
    except:
        time.sleep(1)
    answers = driver.find_elements_by_class_name('answer-content__item._contentWrap._answer')
    for ans in answers:
        ps = ans.find_elements_by_tag_name('p')
        for p in ps:
            answer_text_list.append(p.text)
driver.close()

100%|██████████| 4/4 [00:12<00:00,  3.10s/it]


In [22]:
len(answer_text_list)

145

In [23]:
df = pd.DataFrame({
    'answer': answer_text_list
})
df.head()

,answer
0,비공개 답변
1,상승장에서 순환매예요.
2,보통주들이 오르고나면 우선주들도 나중에 테마를 형성해서 한번씩 급등해요.
3,
4,


In [24]:
today = datetime.today().strftime("%y%m%d")
df.to_csv(f'./data/{KEYWORD}_{today}_org.txt', sep='\t', index=False)

### 데이터 전처리

In [32]:
df_test = pd.read_table(f'./data/{KEYWORD}_{today}_org.txt')
df_test.head()

,answer
0,비공개 답변
1,상승장에서 순환매예요.
2,보통주들이 오르고나면 우선주들도 나중에 테마를 형성해서 한번씩 급등해요.
3,NaN
4,NaN


In [33]:
df_test.shape

(145, 1)

In [34]:
# 한글, 영어 제외한 문자 제거
df_test['answer'] = df_test['answer'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_test.head()

<ipython-input-34-221eee7c9aaa>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['answer'] = df_test['answer'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,answer
0,비공개 답변
1,상승장에서 순환매예요
2,보통주들이 오르고나면 우선주들도 나중에 테마를 형성해서 한번씩 급등해요
3,NaN
4,NaN


In [35]:
# 공백 여부 확인
drop_idx = df_test[df_test.answer == ""].index
print(len(drop_idx), drop_idx)

0 Int64Index([], dtype='int64')


In [36]:
# null값 여부 확인
df_test.isnull().sum()

answer    41
dtype: int64

In [37]:
# null값을 가지고 있는 행 제거
df_test.dropna(axis=0, inplace=True)
df_test.shape

(104, 1)

In [38]:
df_test.head()

,answer
0,비공개 답변
1,상승장에서 순환매예요
2,보통주들이 오르고나면 우선주들도 나중에 테마를 형성해서 한번씩 급등해요
5,제 비트코인으로 유명한 파이코인 비코인
6,스마트폰만 있으면 하루 한번 클릭으로


In [39]:
df_test.to_csv(f'./result/{KEYWORD}_{today}_res.txt', sep='\t', index=False)